# Prompt, Mask, Train, Save, Pipeline-Predict

#### prompt template: 2

# Notebook:

##### Imports
##### replace 'MajorClaim' by 'Stance'
##### make train and test datasets
##### train BERT Mask on the train dataset
##### save the model
##### create a pipeline using the saved model
##### run predictions on it
##### evaluate with the test set

### Imports

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install pandas
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
import torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM

import numpy as np

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

In [4]:
import pandas as pd

### tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### make the train, test dataframe

In [6]:
df = pd.read_pickle("/notebooks/Prompting/notebooks/pe_dataset_w_prompts_1_2_3_pickle")

In [7]:
# rename MajorClaim as stance

In [8]:
df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
5970,essay402,T13,Premise 1393 1436,they can contribute positively to communi

In [9]:
def rename_MC(x):
    
    prompted_rep = x.prompted_representation_2
    
    last_token = prompted_rep.split()[-1]
    
    if last_token == 'MajorClaim.':
    
        new_rep = prompted_rep.replace(last_token, "Stance.")
        
    else:
        
        new_rep = prompted_rep
    
    return new_rep
    
    

In [10]:
rename_MC(df.iloc[0])

'Which of these choices best describes the following component? "MajorClaim", "Claim" or "Premise". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Stance.'

In [11]:
# correct now. now run it on the whole df.

In [12]:
df['prompted_rep_2_w_modif_MC'] = df.apply(lambda x: rename_MC(x), axis=1)

In [13]:
df['prompted_rep_2_w_modif_MC'][0]

'Which of these choices best describes the following component? "MajorClaim", "Claim" or "Premise". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Stance.'

In [14]:
# correct the major claim? --> stance business

In [15]:
def replace_mc(x):
    
    sent = x.prompted_rep_2_w_modif_MC
    new_sent = sent.replace('the following component? "MajorClaim"', 'the following component? "Stance"')
    
    return new_sent

In [16]:
df['prompted_rep_2_w_modif_MC_corrected'] = df.apply(lambda x: replace_mc(x), axis=1)

In [17]:
df['prompted_rep_2_w_modif_MC_corrected'][0]

'Which of these choices best describes the following component? "Stance", "Claim" or "Premise". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Stance.'

In [18]:
# now do a sanity check to see if it's correct

In [19]:
last_tokens_list = []

In [20]:
def get_last_token(x):
    
    prompted_rep = x.prompted_rep_2_w_modif_MC_corrected
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_list.append(last_token)

In [21]:
df.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5968    None
5969    None
5970    None
5971    None
5972    None
Length: 5973, dtype: object

In [22]:
last_tokens_list

['Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',


In [23]:
set(last_tokens_list)

{'Claim.', 'Premise.', 'Stance.'}

In [24]:
####### YAAAAAAAAASSSSSSSSSS KINGGGGGGGGGGGGGGGG # correct.

## make train, test datasets

### train dataset

In [25]:
df_train = df[df.split == 'TRAIN']

In [26]:
df_train = df_train.reset_index(drop=True)

In [27]:
df_train

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_rep_2_w_modif_MC,prompted_rep_2_w_modif_MC_corrected
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4708,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
4709,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, V

In [28]:
df_train.split.value_counts()

TRAIN    4713
Name: split, dtype: int64

In [29]:
prompted_texts_train_full = df_train['prompted_rep_2_w_modif_MC_corrected'][:].tolist()

In [30]:
len(prompted_texts_train_full)

4713

In [31]:
4713 * .2

942.6

In [32]:
import random

In [33]:
prompted_texts_val = random.sample(prompted_texts_train_full, 472)

In [34]:
len(prompted_texts_val)

472

In [35]:
prompted_texts_train = [x for x in prompted_texts_train_full if x not in prompted_texts_val]

In [36]:
len(prompted_texts_train)

4241

In [37]:
len(prompted_texts_train) + len(prompted_texts_val)

4713

In [38]:
### prepare train dataset

In [39]:
inputs_train = tokenizer(prompted_texts_train, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [40]:
inputs_train

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [41]:
inputs_train['input_ids'].shape

torch.Size([4241, 512])

In [42]:
# for training we will need labels

In [43]:
inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [44]:
inputs_train

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
      

In [45]:
inputs_train['labels'].shape

torch.Size([4241, 512])

In [46]:
# find where the last 102 is

In [47]:
(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

126

In [48]:
tokenizer.decode(4366)

'c l a i m'

In [49]:
tokenizer.decode(18458)

'p r e m i s e'

In [50]:
# claim = 4366, premise = 18458

In [51]:
(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

151

In [52]:
inputs_train['input_ids'][1][151]

tensor(102)

In [53]:
# confirm that all at [len-2] indices are either claim or premise

In [54]:
sep_indices_list = []

for i in range(len(prompted_texts_train)):
    sep_idx = (inputs_train['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [55]:
sep_indices_list

[126,
 151,
 150,
 176,
 225,
 155,
 106,
 135,
 157,
 293,
 119,
 145,
 159,
 148,
 118,
 193,
 144,
 129,
 154,
 124,
 134,
 181,
 147,
 140,
 167,
 148,
 110,
 145,
 184,
 156,
 136,
 124,
 213,
 146,
 133,
 130,
 136,
 226,
 212,
 166,
 158,
 141,
 124,
 129,
 162,
 132,
 137,
 149,
 185,
 175,
 153,
 175,
 148,
 175,
 170,
 109,
 126,
 112,
 115,
 97,
 116,
 138,
 160,
 114,
 136,
 105,
 129,
 103,
 133,
 136,
 161,
 107,
 131,
 145,
 143,
 133,
 182,
 117,
 144,
 156,
 157,
 138,
 148,
 120,
 131,
 110,
 118,
 162,
 168,
 122,
 104,
 121,
 135,
 165,
 135,
 123,
 107,
 120,
 131,
 101,
 135,
 236,
 220,
 107,
 125,
 151,
 150,
 149,
 175,
 124,
 128,
 121,
 124,
 107,
 147,
 106,
 100,
 191,
 138,
 159,
 135,
 137,
 139,
 133,
 155,
 158,
 144,
 132,
 154,
 110,
 154,
 121,
 124,
 160,
 109,
 103,
 110,
 152,
 148,
 170,
 151,
 156,
 155,
 121,
 201,
 142,
 109,
 150,
 150,
 111,
 182,
 141,
 150,
 131,
 133,
 179,
 132,
 143,
 194,
 114,
 124,
 135,
 214,
 129,
 152,
 123,
 105,

In [56]:
len(sep_indices_list)

4241

In [57]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [58]:
class_tokens_indices_list

[124,
 149,
 148,
 174,
 223,
 153,
 104,
 133,
 155,
 291,
 117,
 143,
 157,
 146,
 116,
 191,
 142,
 127,
 152,
 122,
 132,
 179,
 145,
 138,
 165,
 146,
 108,
 143,
 182,
 154,
 134,
 122,
 211,
 144,
 131,
 128,
 134,
 224,
 210,
 164,
 156,
 139,
 122,
 127,
 160,
 130,
 135,
 147,
 183,
 173,
 151,
 173,
 146,
 173,
 168,
 107,
 124,
 110,
 113,
 95,
 114,
 136,
 158,
 112,
 134,
 103,
 127,
 101,
 131,
 134,
 159,
 105,
 129,
 143,
 141,
 131,
 180,
 115,
 142,
 154,
 155,
 136,
 146,
 118,
 129,
 108,
 116,
 160,
 166,
 120,
 102,
 119,
 133,
 163,
 133,
 121,
 105,
 118,
 129,
 99,
 133,
 234,
 218,
 105,
 123,
 149,
 148,
 147,
 173,
 122,
 126,
 119,
 122,
 105,
 145,
 104,
 98,
 189,
 136,
 157,
 133,
 135,
 137,
 131,
 153,
 156,
 142,
 130,
 152,
 108,
 152,
 119,
 122,
 158,
 107,
 101,
 108,
 150,
 146,
 168,
 149,
 154,
 153,
 119,
 199,
 140,
 107,
 148,
 148,
 109,
 180,
 139,
 148,
 129,
 131,
 177,
 130,
 141,
 192,
 112,
 122,
 133,
 212,
 127,
 150,
 121,
 103,
 

In [59]:
# inputs_pe['input_ids'][0][143].item() # correct

In [60]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [61]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [62]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [63]:
tokenizer.decode(11032)

's t a n c e'

In [64]:
# correct

In [65]:
# now make the class token indices 103

In [66]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [67]:
# now check if they are 103

In [68]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [69]:
set(unsontroi_check_list)

{103}

In [70]:
# correct

In [71]:
class PeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [72]:
train_dataset = PeDataset(inputs_train)

In [73]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=)
# maybe we don't need this.

### prepare val dataset

In [74]:
inputs_eval = tokenizer(prompted_texts_val, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [75]:
inputs_eval

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [76]:
inputs_eval['input_ids'].shape

torch.Size([472, 512])

In [77]:
# for training we will need labels

In [78]:
inputs_eval['labels'] = inputs_eval.input_ids.detach().clone()

In [79]:
inputs_eval

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
      

In [80]:
inputs_eval['labels'].shape

torch.Size([472, 512])

In [81]:
# find where the last 102 is

In [82]:
(inputs_eval['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

144

In [83]:
tokenizer.decode(4366)

'c l a i m'

In [84]:
tokenizer.decode(18458)

'p r e m i s e'

In [85]:
# claim = 4366, premise = 18458

In [86]:
(inputs_eval['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

163

In [87]:
inputs_eval['input_ids'][1][123]

tensor(1010)

In [88]:
# confirm that all at [len-2] indices are either claim or premise

In [89]:
sep_indices_list = []

for i in range(len(prompted_texts_val)):
    sep_idx = (inputs_eval['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [90]:
sep_indices_list

[144,
 163,
 114,
 154,
 105,
 129,
 136,
 152,
 127,
 117,
 144,
 121,
 116,
 120,
 153,
 131,
 133,
 158,
 157,
 115,
 195,
 161,
 121,
 185,
 135,
 140,
 134,
 122,
 184,
 118,
 185,
 146,
 110,
 122,
 137,
 130,
 163,
 166,
 146,
 123,
 176,
 139,
 147,
 121,
 114,
 129,
 165,
 117,
 191,
 145,
 136,
 113,
 135,
 125,
 117,
 185,
 177,
 140,
 190,
 189,
 126,
 156,
 102,
 127,
 112,
 159,
 195,
 130,
 153,
 147,
 103,
 148,
 200,
 128,
 178,
 169,
 161,
 148,
 135,
 128,
 142,
 185,
 138,
 129,
 130,
 148,
 126,
 152,
 119,
 181,
 213,
 113,
 127,
 183,
 130,
 157,
 164,
 129,
 166,
 128,
 113,
 164,
 188,
 187,
 105,
 146,
 130,
 167,
 198,
 147,
 122,
 128,
 134,
 139,
 150,
 150,
 166,
 153,
 133,
 119,
 127,
 122,
 129,
 109,
 105,
 146,
 161,
 237,
 115,
 174,
 205,
 154,
 137,
 151,
 111,
 114,
 136,
 114,
 132,
 125,
 181,
 195,
 141,
 114,
 111,
 162,
 117,
 164,
 191,
 145,
 131,
 129,
 164,
 112,
 146,
 129,
 132,
 161,
 180,
 194,
 122,
 204,
 176,
 130,
 140,
 155,
 117

In [91]:
len(sep_indices_list)

472

In [92]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [93]:
class_tokens_indices_list

[142,
 161,
 112,
 152,
 103,
 127,
 134,
 150,
 125,
 115,
 142,
 119,
 114,
 118,
 151,
 129,
 131,
 156,
 155,
 113,
 193,
 159,
 119,
 183,
 133,
 138,
 132,
 120,
 182,
 116,
 183,
 144,
 108,
 120,
 135,
 128,
 161,
 164,
 144,
 121,
 174,
 137,
 145,
 119,
 112,
 127,
 163,
 115,
 189,
 143,
 134,
 111,
 133,
 123,
 115,
 183,
 175,
 138,
 188,
 187,
 124,
 154,
 100,
 125,
 110,
 157,
 193,
 128,
 151,
 145,
 101,
 146,
 198,
 126,
 176,
 167,
 159,
 146,
 133,
 126,
 140,
 183,
 136,
 127,
 128,
 146,
 124,
 150,
 117,
 179,
 211,
 111,
 125,
 181,
 128,
 155,
 162,
 127,
 164,
 126,
 111,
 162,
 186,
 185,
 103,
 144,
 128,
 165,
 196,
 145,
 120,
 126,
 132,
 137,
 148,
 148,
 164,
 151,
 131,
 117,
 125,
 120,
 127,
 107,
 103,
 144,
 159,
 235,
 113,
 172,
 203,
 152,
 135,
 149,
 109,
 112,
 134,
 112,
 130,
 123,
 179,
 193,
 139,
 112,
 109,
 160,
 115,
 162,
 189,
 143,
 129,
 127,
 162,
 110,
 144,
 127,
 130,
 159,
 178,
 192,
 120,
 202,
 174,
 128,
 138,
 153,
 115

In [94]:
# inputs_pe['input_ids'][0][143].item() # correct

In [95]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [96]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [97]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [98]:
tokenizer.decode(11032)

's t a n c e'

In [99]:
# correct

In [100]:
# now make the class token indices 103

In [101]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_eval['input_ids'][idx][val] = 103

In [102]:
# now check if they are 103

In [103]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [104]:
set(unsontroi_check_list)

{103}

In [105]:
# correct

In [106]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [107]:
eval_dataset = PeDataset(inputs_eval)

### test dataset

In [108]:
df_test = df[df.split == 'TEST']

In [109]:
df_test = df_test.reset_index(drop=True)

In [110]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_rep_2_w_modif_MC,prompted_rep_2_w_modif_MC_corrected
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not

In [111]:
df_test.split.value_counts()

TEST    1260
Name: split, dtype: int64

In [112]:
prompted_texts_test = df_test['prompted_rep_2_w_modif_MC_corrected'][:].tolist()

In [113]:
len(prompted_texts_test)

1260

In [114]:
inputs_test = tokenizer(prompted_texts_test, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [115]:
inputs_test

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [116]:
inputs_test['input_ids'].shape

torch.Size([1260, 512])

In [117]:
# for test we do not need labels

In [118]:
#inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [119]:
#inputs_train

In [120]:
#inputs_train['labels'].shape

In [121]:
# find where the last 102 is

In [122]:
#(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

In [123]:
#tokenizer.decode(4366)

In [124]:
#tokenizer.decode(18458)

In [125]:
# claim = 4366, premise = 18458

In [126]:
#(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

In [127]:
#inputs_train['input_ids'][1][169]

In [128]:
# confirm that all at [len-2] indices are either claim or premise

In [129]:
sep_indices_list_test = []

for i in range(len(prompted_texts_test)):
    sep_idx = (inputs_test['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list_test.append(sep_idx)

In [130]:
sep_indices_list_test

[151,
 133,
 134,
 117,
 104,
 143,
 259,
 150,
 128,
 236,
 155,
 120,
 146,
 159,
 142,
 145,
 205,
 152,
 136,
 111,
 118,
 119,
 122,
 99,
 146,
 133,
 139,
 143,
 110,
 139,
 111,
 124,
 127,
 113,
 149,
 140,
 108,
 143,
 106,
 132,
 132,
 116,
 119,
 127,
 120,
 99,
 107,
 123,
 118,
 93,
 103,
 166,
 128,
 149,
 109,
 141,
 156,
 146,
 108,
 120,
 147,
 150,
 153,
 132,
 163,
 143,
 196,
 179,
 225,
 170,
 170,
 144,
 124,
 144,
 117,
 133,
 148,
 115,
 137,
 143,
 127,
 152,
 160,
 150,
 136,
 108,
 114,
 146,
 160,
 107,
 104,
 111,
 124,
 134,
 124,
 161,
 137,
 151,
 121,
 129,
 111,
 187,
 100,
 176,
 147,
 143,
 145,
 133,
 121,
 231,
 140,
 152,
 148,
 113,
 135,
 187,
 152,
 137,
 116,
 128,
 135,
 165,
 193,
 134,
 186,
 102,
 204,
 102,
 157,
 116,
 162,
 130,
 140,
 168,
 118,
 161,
 115,
 134,
 99,
 132,
 168,
 117,
 136,
 112,
 142,
 165,
 201,
 119,
 118,
 124,
 147,
 185,
 134,
 133,
 130,
 137,
 156,
 112,
 152,
 163,
 212,
 143,
 120,
 130,
 150,
 144,
 120,
 1

In [131]:
len(sep_indices_list_test)

1260

In [132]:
class_tokens_indices_test_list = [x - 2 for x in sep_indices_list_test]

In [133]:
class_tokens_indices_test_list

[149,
 131,
 132,
 115,
 102,
 141,
 257,
 148,
 126,
 234,
 153,
 118,
 144,
 157,
 140,
 143,
 203,
 150,
 134,
 109,
 116,
 117,
 120,
 97,
 144,
 131,
 137,
 141,
 108,
 137,
 109,
 122,
 125,
 111,
 147,
 138,
 106,
 141,
 104,
 130,
 130,
 114,
 117,
 125,
 118,
 97,
 105,
 121,
 116,
 91,
 101,
 164,
 126,
 147,
 107,
 139,
 154,
 144,
 106,
 118,
 145,
 148,
 151,
 130,
 161,
 141,
 194,
 177,
 223,
 168,
 168,
 142,
 122,
 142,
 115,
 131,
 146,
 113,
 135,
 141,
 125,
 150,
 158,
 148,
 134,
 106,
 112,
 144,
 158,
 105,
 102,
 109,
 122,
 132,
 122,
 159,
 135,
 149,
 119,
 127,
 109,
 185,
 98,
 174,
 145,
 141,
 143,
 131,
 119,
 229,
 138,
 150,
 146,
 111,
 133,
 185,
 150,
 135,
 114,
 126,
 133,
 163,
 191,
 132,
 184,
 100,
 202,
 100,
 155,
 114,
 160,
 128,
 138,
 166,
 116,
 159,
 113,
 132,
 97,
 130,
 166,
 115,
 134,
 110,
 140,
 163,
 199,
 117,
 116,
 122,
 145,
 183,
 132,
 131,
 128,
 135,
 154,
 110,
 150,
 161,
 210,
 141,
 118,
 128,
 148,
 142,
 118,
 14

In [134]:
# inputs_pe['input_ids'][0][143].item() # correct

In [135]:
# now get a list of input ids at all these indices: they should be either claim = 4366, premise = 18458

In [136]:
list_index_minus_2_test = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    list_index_minus_2_test.append(at_idx_minus_2)

In [137]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [138]:
# correct

In [139]:
# now make the class token indices 103

In [140]:
for idx, val in enumerate(class_tokens_indices_test_list):
    inputs_test['input_ids'][idx][val] = 103

In [141]:
# now check if they are 103

In [142]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [143]:
set(unsontroi_check_list)

{103}

In [144]:
# correct

In [145]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [146]:
test_dataset = PeDataset(inputs_test)

In [147]:
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
# maybe we don't need this.

## get the model

In [148]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [149]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [150]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [151]:
# let's see with a simple trainer now and then we will try the fancy trainer.

### training

In [152]:
from transformers import TrainingArguments

In [153]:
import gc

In [154]:
gc.collect()
torch.cuda.empty_cache()

In [155]:
NB_EPOCHS = 10
BATCH_SIZE = 8

In [156]:
training_args = TrainingArguments(
    
    # output
    output_dir='output',          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="epoch",              # cf. paper Sun et al.
    #eval_steps=20,                            # cf. paper Sun et al.
    #eval_accumulation_steps = BATCH_SIZE,
    # fp16_full_eval = True,
    
    # clean the tensor board folder to see the loss properly.
    
    # log
    logging_dir="/notebooks/Prompting/results",  
    logging_strategy='steps',
    logging_steps=100,
    
    # save
    save_strategy='epoch',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss' 
    # metric_for_best_model='f1'
)

In [157]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir = 'out',
#     per_device_train_batch_size=16,
#     num_train_epochs=2
# )

In [158]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
    #compute_metrics=compute_metrics
)

In [159]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4241
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5310
/tmp/ipykernel_196/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.009800,0.004312
2,0.002100,0.001359
3,0.001400,0.001189
4,0.001100,0.001260
5,0.001100,0.001074
6,0.001100,0.001082
7,0.001000,0.001244
8,0.000900,0.000974
9,0.000800,0.001003
10,0.000900,0.000981


***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-531
Configuration saved in output/checkpoint-531/config.json
Model weights saved in output/checkpoint-531/pytorch_model.bin
/tmp/ipykernel_196/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-1062
Configuration saved in output/checkpoint-1062/config.json
Model weights saved in output/checkpoint-1062/pytorch_model.bin
/tmp/ipykernel_196/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t

TrainOutput(global_step=5310, training_loss=0.25779909381446503, metrics={'train_runtime': 2919.4553, 'train_samples_per_second': 14.527, 'train_steps_per_second': 1.819, 'total_flos': 1.1162516110848e+16, 'train_loss': 0.25779909381446503, 'epoch': 10.0})

In [160]:
trainer.save_model("pe_mask_model_prompted_rep_2")

Saving model checkpoint to pe_mask_model_prompted_rep_2
Configuration saved in pe_mask_model_prompted_rep_2/config.json
Model weights saved in pe_mask_model_prompted_rep_2/pytorch_model.bin


In [156]:
# new idea: take this model and finetune the PE dataset on it.
# so instead of loading a pre-trained BERT, load this instead.

### Pipeline

In [161]:
saved_model = BertForMaskedLM.from_pretrained("/notebooks/Prompting/notebooks/prompt_template_2/pe_mask_model_prompted_rep_2")

loading configuration file /notebooks/Prompting/notebooks/prompt_template_2/pe_mask_model_prompted_rep_2/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /notebooks/Prompting/notebooks/prompt_template_2/pe_mask_model_prompted_rep_2/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

Al

In [162]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [163]:
from transformers import pipeline

In [164]:
classifier = pipeline("fill-mask", model=saved_model, tokenizer=tokenizer)

In [165]:
classifier.model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [166]:
targets = ['stance', 'claim', 'premise']

In [167]:
# ok now see how to do this for the whole list of test components

In [168]:
# but first we must replace the last token by [MASK]!

In [170]:
def get_masked_components(x):
    
    prompted_rep = x.prompted_rep_2_w_modif_MC_corrected
    
    last_token = prompted_rep.split()[-1]
    

    new_rep = prompted_rep.replace(last_token, "[MASK].")

    
    return new_rep

In [171]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_rep_2_w_modif_MC,prompted_rep_2_w_modif_MC_corrected
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not

In [172]:
df_test['masked_representation_2_for_pipeline'] = df_test.apply(lambda x: get_masked_components(x), axis=1)

In [173]:
df_test['masked_representation_2_for_pipeline'][0]

'Which of these choices best describes the following component? "Stance", "Claim" or "Premise". Component: Topic: International tourism is now more common than ever before, Sentence: While some might think the tourism bring large profit for the destination countries, I would contend that this industry has affected the cultural attributes and damaged the natural environment of the tourist destinations., First or last in essay: Yes, First in paragraph: No, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: DET, NOUN, AUX, VERB, DET, ADJ, NOUN, CCONJ, VERB, DET, ADJ, NOUN, ADP, DET, NOUN, NOUN. [MASK].'

In [174]:
# ok now do a sanity check to see that they are done correctly

In [175]:
last_tokens_mask_list = []

In [178]:
def get_last_token(x):
    
    prompted_rep = x.masked_representation_2_for_pipeline
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_mask_list.append(last_token)

In [179]:
df_test.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1255    None
1256    None
1257    None
1258    None
1259    None
Length: 1260, dtype: object

In [180]:
last_tokens_mask_list

['[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[M

In [181]:
set(last_tokens_mask_list)

{'[MASK].'}

In [182]:
###### YASSSSS SIRRRRRRRRRRR # correct

In [169]:
# ok now see how we can give all these 1260 statements to the pipeline

In [183]:
test_components_list = df_test['masked_representation_2_for_pipeline'][:].tolist()

In [184]:
len(test_components_list)

1260

In [185]:
# ok now give it to the pipeline

In [186]:
classifier_output = classifier(test_components_list, targets=targets)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [187]:
classifier_output[0][0]['token']

11032

In [188]:
predicted_labels_test = []

for idx in range(1260):
    
    predicted_token = classifier_output[idx][0]['token']
    predicted_labels_test.append(predicted_token)

In [189]:
len(predicted_labels_test)

1260

In [190]:
# sanity check
set(predicted_labels_test)

{4366, 11032, 18458}

In [191]:
# ok correct

In [192]:
test_labels_str = df_test.label_ComponentType[:].tolist()

In [193]:
test_labels_str

['MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Claim',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Premise',
 'Premise',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',

In [194]:
set(test_labels_str)

{'Claim', 'MajorClaim', 'Premise'}

In [195]:
test_labels_int = []

In [196]:
def get_int_labels():
    
    for idx, val in enumerate(test_labels_str):
        
        if val == 'MajorClaim':
            
            test_labels_int.append(11032)
            
        elif val == 'Claim':
            
            test_labels_int.append(4366)
            
        elif val == 'Premise':
            
            test_labels_int.append(18458)

In [197]:
get_int_labels()

In [198]:
test_labels_int

[11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 11032,
 4366,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 18458,
 18458,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 4366,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 11032,


In [199]:
len(test_labels_int)

1260

In [200]:
set(test_labels_int)

{4366, 11032, 18458}

In [201]:
# now do the classification report

In [202]:
from sklearn.metrics import classification_report

In [203]:
print(classification_report(test_labels_int, predicted_labels_test, digits=3))

              precision    recall  f1-score   support

        4366      0.668     0.685     0.676       302
       11032      0.767     0.948     0.848       153
       18458      0.928     0.877     0.902       805

    accuracy                          0.840      1260
   macro avg      0.788     0.837     0.809      1260
weighted avg      0.846     0.840     0.841      1260

